In [54]:
!pip -qq install --upgrade youtube-dl
!pip -qq install git+https://github.com/openai/whisper.git 
!pip -qq install transformers
!pip -qq install sentencepiece
!pip -qq install spacy-transformers

In [3]:
import random
import json 
import pandas as pd
import os
from tqdm.notebook import tqdm

# Remplacez la ligne de code dépréciée par la suivante

import torch
import time
from os.path import exists as path_exists
from pathlib import Path

#Spacy 
import spacy
from spacy.tokens import DocBin
from spacy.util import minibatch
from spacy.training import Example
from spacy.scorer import Scorer
from spacy.lang.fr import French     
from spacy.pipeline import EntityRuler

#huggingface
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AutoModelForSeq2SeqLM

#whisper
import whisper

In [4]:
torch.cuda.is_available()

True

In [10]:
print(os.getcwd())
df = pd.read_csv("/home/echo/Documents/Echo-projects/Alexandre/anamnese/content/archive.zip", compression='zip', header=0, sep=',', quotechar='"')
df.head()     

/home/echo/Documents/Echo-projects/Alexandre/anamnese


,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


ICI CHANGER LE HELSINKI MODEL POUR PASSER D L ESPAGNOL AU FRANCAIS §§§§§§§§§§§§§§§§§§§§§§§§§§§§§§   

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_t = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

model_t = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
  

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_t = model_t.to(device)

NameError: name 'model_t' is not defined

In [12]:
def translate(text: str):
    inputs = tokenizer_t.encode(text, return_tensors="pt", padding=True, max_length=512, truncation=True)
    inputs = inputs.to(device)
    outputs = model_t.generate(inputs, max_length=512, num_beams=5, early_stopping=True)

    decoded_text = tokenizer_t.decode(outputs[0])
    cleaned_text = tokenizer_t.decode(outputs[0], skip_special_tokens=True) ## GPT
    clean_text = tokenizer_t.decode(outputs[0]).replace('',"").replace('',"").replace('',"").strip().lower()
    return cleaned_text

     

In [6]:
def translate(text:str):
  inputs = tokenizer_t.encode( text, return_tensors="pt",padding=True,max_length=512,truncation=True)
  inputs = inputs.to(device)
  outputs = model_t.generate(inputs, max_length=512, num_beams=5, early_stopping=True)
  translated = tokenizer_t.decode(outputs[0], skip_special_tokens=True) ## GPT
  max_length = 512  # Longueur maximale souhaitée
  
  if len(translated) > max_length:
    # Calculez combien de caractères à conserver au début et à la fin
    chars_to_keep_at_start = max_length // 2
    chars_to_keep_at_end = max_length - chars_to_keep_at_start

    # Extraire les premiers caractères et les derniers caractères pour respecter la limite
    translated = translated[:chars_to_keep_at_start] + translated[-chars_to_keep_at_end:]

  #delete les #
  return translated

In [152]:
def translate(text:str):
  inputs = tokenizer_t.encode( text, return_tensors="pt",padding=True,max_length=512,truncation=True)
  inputs = inputs.to(device)
  outputs = model_t.generate( inputs, max_length=512, num_beams=None, early_stopping=True)
  translated = tokenizer_t.decode(outputs[0]).replace('',"").replace('',"").replace('',"").strip().lower()

  return translated

In [9]:
"""def translate(text: str):
    inputs = tokenizer_t.encode(text, return_tensors="pt",padding=True,max_length=512,truncation=True)
    inputs = inputs.to(device)
    outputs = model_t.generate(inputs,max_length=512, num_beams=5,early_stopping=True)## num_beams=None ## précédent code 
    #return tokenizer_t.decode(outputs[0], skip_special_tokens=True) ## GPT 
    translated = tokenizer_t.decode(outputs[0]).replace('',"").replace('',"").replace('',"").strip().lower()


    return translated"""

In [11]:
df.tail()['description']

4994     Patient having severe sinusitis about two to ...
4995     This is a 14-month-old baby boy Caucasian who...
4996     A female for a complete physical and follow u...
4997     Mother states he has been wheezing and coughing.
4998     Acute allergic reaction, etiology uncertain, ...
Name: description, dtype: object

In [12]:
df.tail()['description'].apply(translate)


4994    Patient ayant une sinusite sévère il y a envir...
4995    Il s'agit d'un enfant de 14 mois Caucasien qui...
4996    Une femelle pour un physique complet et un sui...
4997              La mère dit qu'il sifflait et toussait.
4998    Réaction allergique aiguë, étiologie incertain...
Name: description, dtype: object

In [13]:
filtered_df = df[['transcription', 'medical_specialty']]
filtered_df.loc[:, 'medical_specialty'] = filtered_df['medical_specialty'].apply(lambda x:str.strip(x))
mask = (filtered_df['medical_specialty'] == 'SOAP / Chart / Progress Notes') | \
       (filtered_df['medical_specialty'] == 'Office Notes') | \
       (filtered_df['medical_specialty'] == 'Consult - History and Phy.') | \
       (filtered_df['medical_specialty'] == 'Emergency Room Reports') | \
       (filtered_df['medical_specialty'] == 'Discharge Summary') | \
       (filtered_df['medical_specialty'] == 'Letters')
filtered_df = filtered_df[~mask]
data_categories  = filtered_df.groupby(filtered_df['medical_specialty'])

filtered_data_categories = data_categories.filter(lambda x:x.shape[0] > 50)
     

In [14]:
filtered_data_categories['medical_specialty'].value_counts()


medical_specialty
Surgery                       1103
Cardiovascular / Pulmonary     372
Orthopedic                     355
Radiology                      273
General Medicine               259
Gastroenterology               230
Neurology                      223
Obstetrics / Gynecology        160
Urology                        158
ENT - Otolaryngology            98
Neurosurgery                    94
Hematology - Oncology           90
Ophthalmology                   83
Nephrology                      81
Pediatrics - Neonatal           70
Pain Management                 62
Psychiatry / Psychology         53
Name: count, dtype: int64

In [16]:
df_cleaned = filtered_data_categories.sample(frac=1.0)
df_cleaned = df_cleaned.dropna(subset=['transcription'])
df_cleaned.head()
     

,transcription,medical_specialty
2761,"CHIEF COMPLAINT: , Transient visual loss lasti...",Neurology
2577,"DIAGNOSIS:,1. Broad-based endocervical poly.,...",Obstetrics / Gynecology
1165,REFERRAL INDICATION AND PREPROCEDURE DIAGNOSES...,Surgery
859,"PREOPERATIVE DIAGNOSIS:, Left carpal tunnel s...",Surgery
3378,"CHIEF COMPLAINT: , Left flank pain and unable ...",General Medicine


Label the NER in English using the huggingface model for biomedical



In [17]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)
     

In [18]:
tqdm.pandas()

df_cleaned['ner_en']  = df_cleaned['transcription'].progress_apply(pipe)
     

  0%|          | 0/3732 [00:00<?, ?it/s]

/home/echo/Documents/Echo-projects/Alexandre/anamnese/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [19]:
df_cleaned.head()


,transcription,medical_specialty,ner_en
2761,"CHIEF COMPLAINT: , Transient visual loss lasti...",Neurology,"[{'entity_group': 'Detailed_description', 'sco..."
2577,"DIAGNOSIS:,1. Broad-based endocervical poly.,...",Obstetrics / Gynecology,"[{'entity_group': 'Detailed_description', 'sco..."
1165,REFERRAL INDICATION AND PREPROCEDURE DIAGNOSES...,Surgery,"[{'entity_group': 'Sign_symptom', 'score': 0.5..."
859,"PREOPERATIVE DIAGNOSIS:, Left carpal tunnel s...",Surgery,"[{'entity_group': 'Biological_structure', 'sco..."
3378,"CHIEF COMPLAINT: , Left flank pain and unable ...",General Medicine,"[{'entity_group': 'Biological_structure', 'sco..."


In [22]:
dict_specialty_keywords = {}
dict_specialty_keywords_fr = {}


def create_dict(list_elem):
    for elem in list_elem:
      if elem['entity_group'] in dict_specialty_keywords:
        dict_specialty_keywords[elem['entity_group']].append(elem['word'])
        dict_specialty_keywords_fr[elem['entity_group']].append(translate(elem['word']))
      else:
        dict_specialty_keywords[elem['entity_group']] = [elem['word']]
        dict_specialty_keywords_fr[elem['entity_group']] = [translate(elem['word'])]
     

In [23]:
tqdm.pandas()

_ = df_cleaned['ner_en'].progress_apply(create_dict)

  0%|          | 0/3732 [00:00<?, ?it/s]

In [24]:
with open("ner_annotations_fr.json", 'w') as f:
    json.dump(dict_specialty_keywords_fr, f) 

In [1]:
import json

def charger_json_dans_dict(fichier):
    try:
        with open(fichier, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"Le fichier '{fichier}' n'a pas été trouvé.")
        return None
    except json.JSONDecodeError:
        print(f"Erreur de décodage JSON dans le fichier '{fichier}'.")
        return None

# Utilisation de la fonction pour charger un fichier JSON dans un dictionnaire
nom_fichier_json = "ner_annotations_fr.json"
dict_specialty_keywords_fr = charger_json_dans_dict(nom_fichier_json)

# dict_specialty_keywords_fr contient maintenant le contenu du fichier JSON


In [20]:
dict_specialty_keywords_fr 
len(dict_specialty_keywords_fr )


38

In [21]:
tqdm.pandas()

df_cleaned['transcription_fr'] = df_cleaned['transcription'].progress_apply(translate)

  0%|          | 0/3732 [00:00<?, ?it/s]

In [22]:
df_cleaned["ner_en"]

2761    [{'entity_group': 'Detailed_description', 'sco...
2577    [{'entity_group': 'Detailed_description', 'sco...
1165    [{'entity_group': 'Sign_symptom', 'score': 0.5...
859     [{'entity_group': 'Biological_structure', 'sco...
3378    [{'entity_group': 'Biological_structure', 'sco...
                              ...                        
2786    [{'entity_group': 'Biological_structure', 'sco...
3377    [{'entity_group': 'Biological_structure', 'sco...
3189    [{'entity_group': 'Disease_disorder', 'score':...
4965    [{'entity_group': 'Age', 'score': 0.975387, 'w...
4716    [{'entity_group': 'Clinical_event', 'score': 0...
Name: ner_en, Length: 3732, dtype: object

KeyError: 'ner_en'

In [23]:
df_cleaned.to_csv("df_cleaned.csv")


In [2]:
import pandas as pd

# Spécifiez le chemin du fichier CSV que vous souhaitez charger
chemin_fichier_csv = "df_cleaned.csv"

# Utilisez la fonction pd.read_csv() pour charger le contenu du fichier CSV dans un DataFrame
df_cleaned = pd.read_csv(chemin_fichier_csv)

# df_cleaned contient maintenant les données du fichier CSV sous forme de DataFrame pandas


Create the annotated dataset


In [4]:
dict_specialty_keywords_fr = json.load(open("ner_annotations_fr.json"))

dict_specialty_keywords_fr.keys()

dict_keys(['Disease_disorder', 'Detailed_description', 'Therapeutic_procedure', 'Diagnostic_procedure', 'Dosage', 'Medication', 'Age', 'Sex', 'Clinical_event', 'Biological_structure', 'Distance', 'Severity', 'Sign_symptom', 'Lab_value', 'Administration', 'Nonbiological_location', 'Date', 'Duration', 'Quantitative_concept', 'Volume', 'Time', 'History', 'Activity', 'Personal_background', 'Subject', 'Coreference', 'Other_event', 'Outcome', 'Other_entity', 'Area', 'Frequency', 'Color', 'Shape', 'Occupation', 'Texture', 'Family_history', 'Qualitative_concept', 'Height'])

In [ ]:

for k in dict_specialty_keywords_fr.keys():
  dict_specialty_keywords_fr[k] = set(dict_specialty_keywords_fr[k])

In [6]:
for k in dict_specialty_keywords_fr.keys():
  dict_specialty_keywords_fr[k] = [el.strip() for el in list(dict_specialty_keywords_fr[k]) if len(el.strip())>=4 or k=="Age"]
     


And delete some wrong words:

In [7]:
for category in dict_specialty_keywords_fr.keys():
    print(category)

Disease_disorder
Detailed_description
Therapeutic_procedure
Diagnostic_procedure
Dosage
Medication
Age
Sex
Clinical_event
Biological_structure
Distance
Severity
Sign_symptom
Lab_value
Administration
Nonbiological_location
Date
Duration
Quantitative_concept
Volume
Time
History
Activity
Personal_background
Subject
Coreference
Other_event
Outcome
Other_entity
Area
Frequency
Color
Shape
Occupation
Texture
Family_history
Qualitative_concept
Height


In [9]:
#for the class sex we delete the following words
#list_to_delete = ['vétéran', 'dentiste' ]
list_to_delete = [ 'mariés']

for el in list_to_delete: 
  dict_specialty_keywords_fr['Sex'].pop(dict_specialty_keywords_fr['Sex'].index(el))

In [10]:
dict_specialty_keywords_fr["History"]

['Capacité',
 'jour',
 'Activités',
 'État',
 'droite',
 'céphalées',
 'sommeil',
 'diabète',
 'hyper',
 '- 41 - semaine',
 'Utilisation',
 'alcool',
 'drogues illicites',
 'droite - main',
 'convulsions',
 'AVC, diabète',
 'hyper',
 "maladie d'alzheimer",
 'collège',
 "consommation d'alcool, de tabac ou de drogues illicites",
 'vêtements propres',
 'une petite serviette sur la tête',
 'sur une roue',
 'à bas',
 'refusé',
 'mariés',
 'adulte',
 "C'est vrai.",
 'Forte',
 'sucré',
 '25 ans',
 'fumer',
 '##co',
 "C'est pas vrai.",
 'droite - main',
 'polysubstance',
 'tabac chronique',
 "dépendance chronique à l'alcool",
 'Tachycardie ventriculaire',
 'aicd',
 'Hépatite',
 'véhicule à moteur',
 'par. 1 - 0 - 0 - 1',
 'relation monogame',
 'hiv infecté',
 'droite - main',
 'hépatite c',
 'drogues illicites',
 'admis',
 '##lipide',
 "N'importe quoi.",
 'hyper',
 "À l'aide",
 'mariés',
 'droite - main',
 '16 ans',
 'éducation',
 'dominante',
 'véhicule',
 "consommer de l'alcool",
 'flexion',

TEMPORAIRE

In [11]:
#for the class History we delete the following words
list_to_delete =[ '25 ans']

for el in list_to_delete: 
  dict_specialty_keywords_fr['History'].pop(dict_specialty_keywords_fr['History'].index(el))
     

In [ ]:
#for the class Nonbiological_location we delete the following words
#list_to_delete = [ 'Dr X']
#for el in list_to_delete: 
#  dict_specialty_keywords_fr['Nonbiological_location'].pop(dict_specialty_keywords_fr['Nonbiological_location'].index(el))
     


ValueError: 'Dr X' is not in list

In [12]:
# for the class Age
list_to_delete = ['seul', 'moyen', 'et', ',', 'à', 'kg', '1812 g', 'école', 'regardant', '## poids', 'ou']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Age'].pop(dict_specialty_keywords_fr['Age'].index(el))
  except:
    continue

In [13]:
list_to_delete = ['Quel est le sens de la bible ?', 'admission', 'vieux', 'couple', 'autres', 'quelques', 'peu après',
 'dedans', 'autres', 'âge 92', 'peut', 'un jour hors de l’école', 'nuit', 'âge de 12 ans', 'les premiers 4', '8 semaines d’âge gestationnel', 'jour de',
 'âge de 42 ans', 'peut', 'cette fois', 'plusieurs', '6 ans', 'vieux', 'âge de 75 ans', '37.5 semaines de gestation', '20 ans', '20 ans', '89 ans', '31 mois', 'vie', 'plus tard dans la journée', 'presque', 'poste', 'certains', 'Quel est le problème ?', 'ceci', 'fait']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Date'].pop(dict_specialty_keywords_fr['Date'].index(el))
  except:
    continue

In [14]:
list_to_delete = ['riche', 'acquiescer', 'tubes', 'L4 - 5', 'voix réelle', 'multiple', 'arrière', 'oui, bien sûr.', 'oui, bien sûr.',
 'processus', 'jambon', '- L4', 'tapis', 'et aînés', 'ajouter', 'courroie', 'court', 'chaque', 'armes',
 'pour', 'droits de l’homme', 'péché', 'basilic', 'et chants latéraux', 'un point', 'L1 - 2', 'faux',
 'repos', 'gomme à mâcher', 'dans le', 'peu', 'bien', 'entre', 'de la', 'un', 'Comme']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Biological_structure'].pop(dict_specialty_keywords_fr['Biological_structure'].index(el))
  except:
    continue
     

In [15]:
list_to_delete = ['dehors !', 'carte', 'dans le', 'solution saline', 'chapitre', 'sombre', 'nuit', 'coton', 'en vrac', 'clair',
 'recherche de mots', 'travaillé', 'dehors', 'photo', 'capable de', 'respiration', 'frappé', 'pertes sur le terrain',
 'tournoyant', 'l’a fait', 'stabilisé', 'L5 -', 'niveau', 'navire', 'force', 'voyage', 'chair', 'fonction',
 'architecture du sommeil', 'jaune', 'actif', 'signal de câble', 'journal', 'dentelle', 'résoudre', 'talc']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Sign_symptom'].pop(dict_specialty_keywords_fr['Sign_symptom'].index(el))
  except:
    continue

In [16]:
list_to_delete = ['peut', 'difficile', 'moins', 'partie', 'comme', 'premier']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Detailed_description'].pop(dict_specialty_keywords_fr['Detailed_description'].index(el))
  except:
    continue



In [17]:
list_to_delete = ['Dans le', 'peut', 'a', 'lieu']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Therapeutic_procedure'].pop(dict_specialty_keywords_fr['Therapeutic_procedure'].index(el))
  except:
    continue

# dict_specialty_keywords_fr['Therapeutic_procedure']

In [18]:

list_to_delete =['pour', 'vivant', 'trois', '- oui.', 'cultivé', 'un 18', 'sept livres', 'carte', 'chambre', 'plus âgé', 'aînés',
                 'âge', 'approuvées', 'début', 'peu', 'fertile', 'quatrième']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Subject'].pop(dict_specialty_keywords_fr['Subject'].index(el))
  except:
    continue


Now, we delete some keys

In [19]:

list_delete = ['Frequency', 'Quantitative_concept',  'Coreference', 'Distance', 'Time', 'Area','Administration',
               'Other_entity', 'Volume', 'Shape', 'Texture', 'Other_event', 'Height','Lab_value', 'Duration']

for elem in list_delete:
  del dict_specialty_keywords_fr[elem] 

In [20]:
dict_specialty_keywords_fr

{'Disease_disorder': ['fibrillation auriculaire',
  'insuffisance respiratoire',
  '##cta',
  'respiratoire',
  'Traumatisme',
  'détresse',
  '##cta',
  'tailleur',
  '##uro',
  '##uro',
  '##gen',
  '##erative',
  'maladie articulaire',
  "C'est pas vrai.",
  'maladie de Parkinson',
  '##yp',
  '# ##oth',
  'acte',
  '##ose',
  'diabète',
  '##odes',
  "Pour l'année",
  '##té',
  'Mon Dieu.',
  'elle',
  'elle',
  'blessure torsadive',
  "l'art",
  'spondylose cervicale',
  'noyau',
  'spondylose',
  '##Maman',
  'régurgitation tricuspide',
  'régurgitation mitrale',
  'cardizem',
  'Carte',
  "Il n'y a pas",
  '##lérose',
  'avec',
  'Robinet ral',
  "C'est vrai.",
  'Maladies sexuellement transmissibles',
  'glioblastome multiforme',
  "J'y vais.",
  'lésions faciales',
  'fosse',
  '##ary aden',
  'carcinome des cellules rénales',
  'ourlet',
  'carcinome à petites cellules',
  '##bulaire',
  'amygdalite',
  'amygdale',
  'amygdalite',
  'amygdale',
  'endométriose',
  'endométrio

Create the ruler ner model



In [21]:
class RulerModel():
    def __init__(self, dict_specialty_keywords):
        self.ruler_model = spacy.blank('fr')
        self.entity_ruler =  self.ruler_model.add_pipe('entity_ruler')
        
        total_patterns = []

        for k,v in dict_specialty_keywords_fr.items():
        
          patterns = self.create_patterns(v, k)
          total_patterns.extend(patterns)
       
        self.add_patterns_into_ruler(total_patterns)
        
       
    def create_patterns(self, entity_type_set, entity_type):
        patterns = []
        for item in entity_type_set:
            pattern = {'label': entity_type, 'pattern': item}
            patterns.append(pattern)
        return patterns
    
    def add_patterns_into_ruler(self, total_patterns):
        self.entity_ruler.add_patterns(total_patterns)


and the dataset generator



In [23]:
class GenerateDataset(object):
    
    def __init__(self, ruler_model):
        self.ruler_model = ruler_model.ruler_model
        
    def find_entitytypes(self, text):
        ents = [] 
        doc = self.ruler_model(str(text))
        for ent in doc.ents:
            ents.append((ent.start_char, ent.end_char, ent.label_))
        return ents     
    
    def assign_labels_to_documents(self, df):
        dataset = []
        text_list = df['transcription_fr'].values.tolist()
        for text in text_list:
            ents = self.find_entitytypes(text)
            if len(ents) > 0:
                dataset.append((text, {'entities': ents}))
            else:
                continue 
        return dataset
     

NameError: name 'df_cleaned' is not defined

In [24]:
ruler_ner = RulerModel(dict_specialty_keywords_fr)
ds_ner = GenerateDataset(ruler_ner)
df_cleaned = pd.read_csv("df_cleaned.csv")
#df_cleaned.rename(columns={'transcription': 'transcription_fr'}, inplace=True) GPT

df_ner = ds_ner.assign_labels_to_documents(df_cleaned)


Let's check the dataset generated:



In [25]:
df_ner[0]


("CHIEF COMPLAINT:, Perte visuelle transitoire ne durant que cinq minutes., HISTORIQUE DE PRESENT ILLNESSE:, C'est un homme blanc très actif et agréable de 82 ans avec un passé médical significatif pour le bloc AV au premier degré, l'état du stimulateur card «comme s'il était comprimé du haut avec une feuille noire qui descendait». L'épisode a duré environ cinq minutes et s'est produit il y a trois semaines pendant qu'il conduisait un véhicule. Il a pu tirer le wagon sur le côté de la route en toute sécurité.",
 {'entities': [(33, 44, 'Detailed_description'),
   (59, 71, 'Detailed_description'),
   (107, 112, 'Diagnostic_procedure'),
   (116, 121, 'Biological_structure'),
   (122, 127, 'Biological_structure'),
   (133, 138, 'Detailed_description'),
   (142, 150, 'Outcome'),
   (154, 160, 'Age'),
   (161, 165, 'Medication'),
   (169, 174, 'Outcome'),
   (175, 182, 'Nonbiological_location'),
   (183, 195, 'Severity'),
   (196, 200, 'Diagnostic_procedure'),
   (204, 208, 'Sign_symptom'),


Training the Spacy NER model
Frist, split in train - test



In [26]:
random.shuffle(df_ner)

train_data = df_ner[:int(len(df_ner)*0.8)]
test_data = df_ner[int(len(df_ner)*0.8):]

And we create the binary dataset as spacy required
to generate train file:



In [27]:
db = DocBin() # create a DocBin object

nlp = spacy.blank('fr') 

for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

  0%|          | 0/2948 [00:00<?, ?it/s]

to generate test file:


In [28]:
db = DocBin() # create a DocBin object

nlp = spacy.blank('fr') 

for text, annot in tqdm(test_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./test.spacy") # save the docbin object

  0%|          | 0/737 [00:00<?, ?it/s]

Go to spacy and generate the case config file, as we show in the image below:
download that base_config file and place in the same diretory



In [29]:
!python -m spacy init fill-config base_config.cfg config.cfg
#!python -m spacy init fill-config base_config-cpu.cfg config.cfg


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

Finally Train the NER model


In [46]:
"""import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
import torch"""

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
torch.cuda.empty_cache()


In [48]:
!python -m spacy train config.cfg --output ./output --paths.train "./train.spacy" --paths.dev "./test.spacy" --gpu-id 0

#!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        1279.68    957.46    0.06    0.08    0.06    0.00
⚠ Aborting and saving the final best model. Encountered exception:
OutOfMemoryError('CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a
total capacty of 5.80 GiB of which 32.69 MiB is free. Process 5609 has 169.41
MiB memory in use. Process 1465092 has 1.16 GiB memory in use. Including
non-PyTorch memory, this process has 4.40 GiB memory in use. Of the allocated
memory 4.00 GiB is allocated by PyTorch, and 258.92 MiB is reserved by PyTorch
but unallocated. If reserved but unallocated 

In [ ]:
#!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy --gpu-id 0


/home/echo/Documents/Echo-projects/Alexandre/anamnese/env/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
ℹ Saving to output directory: output
ℹ Using GPU: 0
Traceback (most recent call last):
  File "/home/echo/.pyenv/versions/3.10.13/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/echo/.pyenv/versions/3.10.13/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/echo/Documents/Echo-projects/Alexandre/anamnese/env/lib/python3.10/site-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/home/echo/Documents/Echo-projects/Alexandre/

Integrating the Whisper ASR


In [35]:
if not path_exists('transcripts'):
    !mkdir transcripts

In [36]:
#YouTubeID = 'xOZM-1p-jAk' # ¿Qué es la enfermedad de Crohn y cómo detectarla?
YouTubeID = '7Ruj4wCx3AQ'
OutputFile = 'test_audio_youtube_5.m4a'

In [ ]:
#if not path_exists(OutputFile):
#    !youtube-dl -o 
#YouTubeID --extract-audio --restrict-filenames -f 'bestaudio[ext=m4a]'
   

SyntaxError: invalid syntax (1901557211.py, line 3)

In [ ]:
"""import os

YouTubeID = "7Ruj4wCx3AQ"
OutputFile = "test_audio_youtube_5.m4a"

if not os.path.exists(OutputFile):
    !youtube-dl --verbose -o "{OutputFile}" "https://www.youtube.com/watch?v={YouTubeID}" --extract-audio --restrict-filenames -f 'bestaudio[ext=m4a]'
"""

[debug] System config: []
[debug] User config: []
[debug] Custom config: []
[debug] Command-line args: ['--verbose', '-o', 'test_audio_youtube_5.m4a', 'https://www.youtube.com/watch?v=7Ruj4wCx3AQ', '--extract-audio', '--restrict-filenames', '-f', 'bestaudio[ext=m4a]']
[debug] Encodings: locale UTF-8, fs utf-8, out utf-8, pref UTF-8
[debug] youtube-dl version 2021.12.17
[debug] Git HEAD: 298815f
[debug] Python version 3.10.13 (CPython) - Linux-6.2.0-37-generic-x86_64-with-glibc2.35
[debug] exe versions: ffmpeg 4.4.2, ffprobe 4.4.2
[debug] Proxy map: {}
[youtube] 7Ruj4wCx3AQ: Downloading webpage
[youtube] 7Ruj4wCx3AQ: Downloading API JSON
ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
Traceback (most recent call last):
  File "/home/echo/Documents/Echo-projects/Alexandre/

In [37]:
OutputFile="/home/echo/Documents/Echo-projects/Alexandre/anamnese/2022_1_4_0_0_24_ch25/2022_1_4_0_0_24_ch25.mp3"

In [38]:
model_whisper = whisper.load_model("small")
result = model_whisper.transcribe(OutputFile)
print(result["text"])

 Bonjour Monsieur. Oui. Oui. Oui. Je vous appelle car j'ai une douleur dans les services calc, c'est affreux, je n'arrive pas à dormir, je tremble. Vous donnez un cas d'arrondissement ou comment? Très bien un arrondissement. Bonne noeud et prénom. Quelle est pas bonne aile. D'adolescence? 1577. D'adresse s'il vous plaît. Encore 61B69. Tout à fait deuxième étage. Interphone le Seine. Oui. Deuxième étage d'auchasse renseignable. Oui. D'accord, vous avez mal ou? J'ai mal d'arrêt service cal. Depuis quand? Ça fait déjà... Là ça va peut-être faire deux jours, trois jours. Et j'arrive pas à dormir et là je tremble, j'ai pas de sieuvre mais je tremble. D'accord. Quelqu'un avec vous? J'ai né deux enfants. D'accord. Pas de douleur à la poitrine? J'ai pas de douleur à la poitrine. Je sais pas. Non j'ai pas de douleur mais j'arrive pas à m'expliquer. Ah vous expliquez quoi? J'ai une douleur dans les services cal que je suis complètement bloqué. D'accord, je vais vous envoyer un message pour vérif

In [39]:
import spacy_transformers
nlp1 = spacy.load(r"/content/output/model-best") #load the best model

doc = nlp1(result["text"])

OSError: [E050] Can't find model '/content/output/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
ents = dict_specialty_keywords_fr.keys()


import random
random.seed(38)

colors = {}
for j in ents:
    rand_colors = ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])]
    colors[j] = rand_colors[0]

options = {"ents": list(ents), "colors": colors }

In [ ]:
spacy.displacy.render(doc, style='ent',jupyter=True, options=options)
